In [60]:
# 라이브러리
import pandas as pd
import numpy as np
import tqdm as tq
import pickle
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from tqdm import tqdm

# 데이터 탐색

In [2]:
train = pd.read_csv('/content/drive/MyDrive/contest/parking_data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/contest/parking_data/test.csv')
age_gender = pd.read_csv('/content/drive/MyDrive/contest/parking_data/age_gender_info.csv')

### 훈련 데이터

In [3]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [4]:
train.corr(method='pearson')

,총세대수,전용면적,전용면적별세대수,공가수,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
총세대수,1.000000,0.042271,0.098205,0.068192,0.213843,-0.001446,0.501488,0.316703
전용면적,0.042271,1.000000,-0.035479,-0.089201,-0.013111,-0.007866,0.071260,0.115453
전용면적별세대수,0.098205,-0.035479,1.000000,0.166472,-0.056680,0.047112,0.281989,0.247085
공가수,0.068192,-0.089201,0.166472,1.000000,-0.093334,0.043285,0.289819,0.118121
도보 10분거리 내 지하철역 수(환승노선 수 반영),0.213843,-0.013111,-0.056680,-0.093334,1.000000,0.098380,-0.055544,-0.108592
도보 10분거리 내 버스정류장 수,-0.001446,-0.007866,0.047112,0.043285,0.098380,1.000000,0.070696,0.103752
단지내주차면수,0.501488,0.071260,0.281989,0.289819,-0.055544,0.070696,1.000000,0.859216
등록차량수,0.316703,0.115453,0.247085,0.118121,-0.108592,0.103752,0.859216,1.000000


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

< Train data null 값 >

임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영), 도보 10분거리 내 버스정류장 수

In [6]:
train.isnull().sum().to_frame()

,0
단지코드,0
총세대수,0
임대건물구분,0
지역,0
공급유형,0
전용면적,0
전용면적별세대수,0
공가수,0
자격유형,0
임대보증금,569


### 테스트 데이터

In [7]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          1022 non-null   object 
 1   총세대수                          1022 non-null   int64  
 2   임대건물구분                        1022 non-null   object 
 3   지역                            1022 non-null   object 
 4   공급유형                          1022 non-null   object 
 5   전용면적                          1022 non-null   float64
 6   전용면적별세대수                      1022 non-null   int64  
 7   공가수                           1022 non-null   float64
 8   자격유형                          1020 non-null   object 
 9   임대보증금                         842 non-null    object 
 10  임대료                           842 non-null    object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  980 non-null    float64
 12  도보 10분거리 내 버스정류장 수            1022 non-null   float64
 13  단지내

In [9]:
test.isnull().sum().to_frame()

,0
단지코드,0
총세대수,0
임대건물구분,0
지역,0
공급유형,0
전용면적,0
전용면적별세대수,0
공가수,0
자격유형,2
임대보증금,180


< Test data null 값 >

자격유형, 임대보증금, 임대료, 도보 10분거리 내 지하철역 수(환승노선 수 반영)

Train, Test null 값을 보면

임대보증금, 임대료는 같이 null값으로 존재할 것으로 생각할 수 있음

# 데이터 전처리

[ https://dacon.io/competitions/official/235745/codeshare/2853?page=1&dtype=recent ] 참고


임대가 아닌 공공분양, 전세 같은 경우는 임대 관련 값은 NA

임대보증금, 임대료를 0으로 대체

In [10]:
# 임대관련 값 '-'을 null로 바꾼 후 float로 타입 변경
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

# 임대관련 값 0으로 대체 
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

임대보증금, 임대료 dtype을 int로 변경

지하철 및 버스정류장 수 관련 Null 값도 없는 것으로 추정하고 0으로 대체

In [11]:
# 지하철 및 버스정류장 수 0으로 대체
col = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[col] = train[col].fillna(0)
test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = test['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].fillna(0)

test data의 자격유형은 최빈값으로 대체 

==> 'A'

In [12]:
test['자격유형'].value_counts() # A가 가장 많음을 확인

A    572
D    180
H     92
J     84
C     34
K     16
L     12
N     10
E     10
I      7
M      2
G      1
Name: 자격유형, dtype: int64

In [13]:
# A로 대체
test['자격유형'] = test['자격유형'].fillna('A')

중복이 상당히 많은 것으로 확인되므로 중복을 제거

In [14]:
train = train.drop_duplicates()
test = test.drop_duplicates()

[ https://dacon.io/competitions/official/235745/codeshare/2851?page=1&dtype=recent ]

train, test의 각 단지코드를 하나의 로우로 합하기

In [15]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

In [16]:
unique_cols = ['총세대수', '지역', '공가수', 
               '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
               '도보 10분거리 내 버스정류장 수',
               '단지내주차면수', '등록차량수']
train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()

In [17]:
train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

In [18]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

In [19]:
train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

In [20]:
train_agg = pd.concat([train_agg,
                       reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')], axis=1)

test_agg = pd.concat([test_agg,
                       reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')], axis=1)

# 모델링

In [22]:
print(train_agg.shape)
print(test_agg.shape)

(423, 20)
(150, 19)


In [49]:
y = train_agg['등록차량수']
x = train_agg.drop(['등록차량수','지역'],axis=1)

In [ ]:
from sklearn.model_selection import GridSearchCV

model = XGBRegressor()

grid = {'n_estimators' : [100], 'eta' : [0.01], 'min_child_weight' : np.arange(1, 8, 1), 
        'max_depth' : np.arange(3,7,1) , 'colsample_bytree' :np.arange(0.8, 1.0, 0.1), 
        'subsample' :np.arange(0.8, 1.0, 0.1)} # fix the n_estimators & eta(learning rate)
        
gcv = GridSearchCV(model, param_grid = grid, cv = 3, refit=True, return_train_score=True)        
    
gcv.fit(x, y)
params = gcv.best_params_

In [63]:
print(gcv.best_estimator_)
pickle.dump(gcv.best_estimator_, open("/content/drive/MyDrive/contest/parking_data/hyperparameter_xgb.pickle", "wb"))

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eta=0.01, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=6, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eta=0.01, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=6, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)

In [73]:
xgb_reg =XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                      colsample_bynode=1, colsample_bytree=0.9, eta=0.01, gamma=0, learning_rate=0.1, max_delta_step=0, 
                      max_depth=3, min_child_weight=6, missing=None, n_estimators=100,
                      n_jobs=1, nthread=None, objective='reg:linear', random_state=0, 
                      reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
                      subsample=0.8, verbosity=1)

In [76]:
target_col = '등록차량수'
xgb_reg.fit(train_agg.drop(columns=[target_col,'지역']), train_agg[target_col])

[05:23:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eta=0.01, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=6, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)

In [77]:
sample_submission = pd.read_csv('/content/drive/MyDrive/contest/parking_data/sample_submission.csv')

In [78]:
predictions = xgb_reg.predict(test_agg.drop(['지역'],axis=1))

In [80]:
predictions

array([ 657.5382  , 1280.7771  ,  563.01227 ,  559.6774  ,  978.154   ,
       1855.462   ,  919.52795 ,  425.94122 ,  371.12762 ,  311.96527 ,
        459.13147 ,  159.27246 ,  385.55014 ,  302.26105 ,  337.11993 ,
        244.02814 ,  473.47363 ,  293.48862 ,  143.85596 ,  775.2272  ,
        191.28717 ,  431.33466 ,  461.6155  ,  550.57935 ,  388.16705 ,
        166.39862 ,   76.060844,  488.55756 ,  411.7519  ,  415.27048 ,
        856.6124  ,  134.13269 ,  478.02094 ,  234.29608 ,  103.68215 ,
        317.16705 ,  431.83133 ,  569.83136 ,  808.31525 ,  325.3884  ,
        454.60464 ,  394.28    ,  374.2597  ,  469.85397 ,  671.3399  ,
       1264.5465  ,  456.5312  ,  582.35004 ,  390.53647 ,  362.42126 ,
        705.055   ,  307.21527 , 1029.1609  ,  675.6119  ,  589.7165  ,
        270.8625  ,  531.40094 ,  316.7731  ,  416.46475 ,  174.76251 ,
        398.9187  ,  488.18643 , 1012.10223 ,  404.7316  ,  168.64473 ,
        280.29724 ,  612.5243  ,  964.384   ,  511.95367 ,  422.

In [82]:
sample_submission['num'] = predictions
sample_submission.head()

,code,num
0,C1072,657.538208
1,C1128,1280.777100
2,C1456,563.012268
3,C1840,559.677429
4,C1332,978.153992


In [84]:
file_name = 'parking04.csv'
sample_submission.to_csv('/content/drive/MyDrive/contest/parking_data/' + file_name, index=False)